# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [202]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag,ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

In [241]:
# load data from database
engine = create_engine('sqlite:///FigureEight.db')
df = pd.read_sql_table('FigureEight',engine)

X = df['message'].values
Y = df[df.columns[3:]].values
lowerword = [w.lower().strip() for w in X]


In [249]:
X[0].lower()

'weather update - a cold front from cuba that could pass over haiti'

In [248]:
detect = re.findall('^a-zA-Z0-9',X[2])
detect

[]

In [239]:
puncword = re.sub('[^a-zA-Z0-9]',' ', lowerword)
puncword

TypeError: expected string or bytes-like object

In [238]:
tokenword = word_tokenize(puncword)
tokenword

TypeError: expected string or bytes-like object

In [46]:
stopword =[]
for word in tokenword:
    stopword.append([w for w in word if w not in stopwords.words('english')])
stopword

[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'],
 ['hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'reports',
  'leogane',
  '80',
  '90',
  'destroyed',
  'hospital',
  'st',
  'croix',
  'functioning',
  'needs',
  'supplies',
  'desperately'],
 ['says', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national', 'palace'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tents', 'water', 'silo', 'thank'],
 ['would', 'like', 'receive', 'messages', 'thank'],
 ['croix',
  'des',
  'bouquets',
  'health',
  'issues',
  'workers',
  'santo',
  '15',
  'area',
  'croix',
  'des',
  'bouquets'],
 ['nothing', 'eat', 'water', 'starving', 'thirsty'],
 ['petionville', 'need', 'information', 'regarding', '4636'],
 ['thomassin',
  'number',
  '32',
  'area',
  'named',
  'pyron',
  'would',
  'like',
  'water',
  'thank',
  'god',
  'fine',
  'desperately',
  'need',
  'water',
  'thanks'],
 ['let', 'together', 'nee

In [65]:
lemmed=[]
for word in stopword:
    lemmed.append([WordNetLemmatizer().lemmatize(w,pos='v') for w in word ])
lemmed

[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'],
 ['hurricane'],
 ['look', 'someone', 'name'],
 ['un',
  'report',
  'leogane',
  '80',
  '90',
  'destroy',
  'hospital',
  'st',
  'croix',
  'function',
  'need',
  'supply',
  'desperately'],
 ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national', 'palace'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tent', 'water', 'silo', 'thank'],
 ['would', 'like', 'receive', 'message', 'thank'],
 ['croix',
  'des',
  'bouquets',
  'health',
  'issue',
  'workers',
  'santo',
  '15',
  'area',
  'croix',
  'des',
  'bouquets'],
 ['nothing', 'eat', 'water', 'starve', 'thirsty'],
 ['petionville', 'need', 'information', 'regard', '4636'],
 ['thomassin',
  'number',
  '32',
  'area',
  'name',
  'pyron',
  'would',
  'like',
  'water',
  'thank',
  'god',
  'fine',
  'desperately',
  'need',
  'water',
  'thank'],
 ['let', 'together', 'need', 'food', 'delma', '7

In [228]:
np.array(lemmed)

array([list(['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']),
       list(['hurricane']), list(['look', 'someone', 'name']), ...,
       list(['proshika', 'operate', 'cox', 'bazar', 'municipality', '5', 'unions', 'ramu', 'chokoria', 'assessment', '5', 'kg', 'rice', '1', '5', 'kg', 'lentils', '700', 'families']),
       list(['2', '000', 'women', 'protest', 'conduct', 'elections', 'teargas', 'try', 'converge', 'local', 'electoral', 'commission', 'offices', 'southern', 'oil', 'city', 'port', 'harcourt']),
       list(['radical', 'shift', 'think', 'come', 'result', 'meet', 'recognize', 'hiv', 'aid', 'core', 'humanitarian', 'crisis', 'identify', 'crisis', 'function', 'hiv', 'aid', 'pandemic'])],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [262]:
def tokenize(text):
    '''
    input: 
        text : message text for processing
    output: 
        tokenized_word : message text after being processed 
    '''
    # Case Normalization
    lower_text = text.lower()
    # Punctutation removal
    ptext =re.sub(r"[^a-zA-Z0-9]","", text)
    # Word Tokenization
    tokenword = word_tokenize(ptext)
    # stopword removal
    stopword = [w for w in tokenword if w not in stopwords.words('english')]
   # tokenized_word = []
    #for word in tokenword:
     #   tokenized_word.append(word if word not in stopwords.words('english')）
    lemmed_word=[]
    for word in stopword:
        lemmed_word.append(WordNetLemmatizer().lemmatize(word,pos='v'))
        
    return lemmed_word

In [263]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer = tokenize)),('tfidf',TfidfTransformer()),
                     ('clf',MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 1))))])
#pipeline

In [264]:
pipeline.fit(X,Y)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                                   

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train 


In [265]:
def train_pipeline(X,Y):
    '''
    input: X,Y used to train classifier
    output: train pipeline

    '''
    
    X_train,X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)
    # train classifier
    return pipeline.fit(X_train, Y_train), X_train, X_test,Y_train,Y_test

   

In [266]:
model, X_train,X_test,Y_train,Y_test = train_pipeline(X,Y)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


In [267]:
Y_predict = model.predict(X_test)
#classification_report(Y_test, Y_predict,target_names=df.columns[3:])
#list(df.columns[3:])

#classification_report(Y_test, Y_predict)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [109]:
def evaluation (test):
    """
    input: testing dataset
    output: evaluation
    """
    

    

NameError: name 'clf' is not defined

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.